In [ ]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
ls

drive/  sample_data/


In [ ]:
cd /content/drive/MyDrive/Minor Project(NLP)

/content/drive/MyDrive/Minor Project(NLP)


In [2]:
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Embedding
from keras.preprocessing import sequence

In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv('./cyberbullying_tweets.csv',encoding = 'ISO-8859-1')

In [ ]:
print(data.head(2))

                                          tweet_text cyberbullying_type
0  In other words #katandandre, your food was cra...  not_cyberbullying
1  Why is #aussietv so white? #MKR #theblock #ImA...  not_cyberbullying


In [ ]:
data['cyberbullying_type'].unique() 

array(['not_cyberbullying', 'gender', 'religion', 'other_cyberbullying',
       'age', 'ethnicity'], dtype=object)

In [ ]:
data['cyberbullying_type'] = data['cyberbullying_type'].replace({'not_cyberbullying':0,'gender':1,'religion':1,'age':1,'ethnicity':1,'other_cyberbullying':1})
print(data['cyberbullying_type'].unique())
X = data['tweet_text'].values
Y = data['cyberbullying_type'].values
print(Y[:2])
print(data.head(2))

[0 1]
[0 0]
                                          tweet_text  cyberbullying_type
0  In other words #katandandre, your food was cra...                   0
1  Why is #aussietv so white? #MKR #theblock #ImA...                   0


In [ ]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download("stopwords")
ps = PorterStemmer()
corpus = []
for i in range(len(X)):
  print(i)
  news = re.sub('[^a-zA-Z]', ' ', X[i])
  news = news.lower()
  news = news.split()
  news = [ps.stem(word) for word in news if word not in stopwords.words('english')]
  news = ' '.join(news)
  corpus.append(news)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


NameError: ignored

In [ ]:
print(X[0],'\n',corpus[0])

In other words #katandandre, your food was crapilicious! #mkr 
 word katandandr food crapilici mkr


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
Xtrain,Xtest,Ytrain,Ytest = train_test_split(corpus,Y,test_size = 0.25,random_state=99)
print(Xtrain[:2])
print(Xtest[:2])
print(Ytrain[:2])
print(Ytest[:2])

['rt fruitondabottom idontneedfemin hate manipul feminist entertain womenagainstfemin letstalkmen h', 'bulli school self explanatori']
['archibaldcran lostsailorni fuck insult mlk wow', 'went high school major black white hispan surpris news racist white cop slam hispan girl ground gay guy school got bulli white kid lacross team']
[1 1]
[1 1]


In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential

In [ ]:
tokenizer = Tokenizer(num_words=5000, lower=True)
tokenizer.fit_on_texts(Xtrain)
train_sequences = tokenizer.texts_to_sequences(Xtrain)
test_sequences = tokenizer.texts_to_sequences(Xtest)

In [ ]:
max_len = 100
train_sequences = pad_sequences(train_sequences, maxlen=max_len)
test_sequences = pad_sequences(test_sequences, maxlen=max_len)

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=32, input_length=max_len))
model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(8,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
model.fit(train_sequences, Ytrain, epochs=100, batch_size=512)


Epoch 1/100
70/70 [==============================] - 1s 9ms/step - loss: 0.0806 - accuracy: 0.9516
Epoch 2/100
70/70 [==============================] - 2s 22ms/step - loss: 0.0797 - accuracy: 0.9513
Epoch 3/100
70/70 [==============================] - 1s 8ms/step - loss: 0.0788 - accuracy: 0.9517
Epoch 4/100
70/70 [==============================] - 1s 14ms/step - loss: 0.0785 - accuracy: 0.9519
Epoch 5/100
70/70 [==============================] - 1s 9ms/step - loss: 0.0777 - accuracy: 0.9525
Epoch 6/100
70/70 [==============================] - 1s 14ms/step - loss: 0.0776 - accuracy: 0.9517
Epoch 7/100
70/70 [==============================] - 1s 12ms/step - loss: 0.0776 - accuracy: 0.9524
Epoch 8/100
70/70 [==============================] - 1s 9ms/step - loss: 0.0764 - accuracy: 0.9533
Epoch 9/100
70/70 [==============================] - 1s 14ms/step - loss: 0.0757 - accuracy: 0.9545
Epoch 10/100
70/70 [==============================] - 1s 9ms/step - loss: 0.0767 - accuracy: 0.9523
Epoc

In [ ]:
# Evaluate the  (binary classification)
test_loss, test_acc = model.evaluate(test_sequences, Ytest, verbose=2)
print('Test accuracy:', test_acc)

373/373 - 1s - loss: 2.9543 - accuracy: 0.8401 - 709ms/epoch - 2ms/step
Test accuracy: 0.8401408791542053


In [ ]:
#10 epochs ----- train acc = 93; test acc = 82.9
#20 epochs ----- train acc = 95; test acc = 83.6
#100 epochs ---- train acc = 95.3 ; test acc = 84

In [ ]:
#CNN model for multiclass classification
multidata = pd.read_csv('./cyberbullying_tweets.csv',encoding='ISO-8859-1')
multidata.head(2)

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying


In [ ]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download("stopwords")
def gen_corpus(X):
  ps = PorterStemmer()
  corpus = []
  for i in range(len(X)):
    print(i)
    news = re.sub('[^a-zA-Z]', ' ', X[i])
    news = news.lower()
    news = news.split()
    news = [ps.stem(word) for word in news if word not in stopwords.words('english')]
    news = ' '.join(news)
    corpus.append(news)
  return corpus

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
multiX = multidata['tweet_text'].values
multidata['cyberbullying_type'] = multidata['cyberbullying_type'].replace({'not_cyberbullying':0,'gender':1,'religion':2,'age':3,'ethnicity':4,'other_cyberbullying':5})
multidata[:2]
mcorpus = gen_corpus(multiX)

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",0
1,Why is #aussietv so white? #MKR #theblock #ImA...,0


In [ ]:
multiXtrain,multiXtest,multiYtrain,multiYtest = train_test_split(mcorpus,multidata['cyberbullying_type'],test_size=0.2,random_state=42)

print(multiYtrain[2])

tokenizer = Tokenizer(num_words=5000, lower=True)
tokenizer.fit_on_texts(multiXtrain)
train_sequences_multi = tokenizer.texts_to_sequences(multiXtrain)
test_sequences_multi = tokenizer.texts_to_sequences(multiXtest)

0


In [ ]:
max_len = 100
train_sequences_multi = pad_sequences(train_sequences_multi, maxlen=max_len)
test_sequences_multi = pad_sequences(test_sequences_multi, maxlen=max_len)

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=32, input_length=max_len))
model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(multiXtrain[:2])
print(multiYtrain[0])

['rt avatast freebsdgirl also anyon never written x c java lt insert languag assign alway ret', 'questionni di joke gay rape babe rape']
0


In [ ]:
# Convert labels to one-hot encoded format
train_labels = tf.keras.utils.to_categorical(multiYtrain, num_classes=6)
test_labels = tf.keras.utils.to_categorical(multiYtest, num_classes=6)

In [ ]:
model.fit(train_sequences_multi, train_labels, epochs=10, batch_size=32)

Epoch 1/10
1193/1193 [==============================] - 27s 21ms/step - loss: 1.4670 - accuracy: 0.3691
Epoch 2/10
1193/1193 [==============================] - 8s 7ms/step - loss: 1.3403 - accuracy: 0.4301
Epoch 3/10
1193/1193 [==============================] - 7s 6ms/step - loss: 1.2913 - accuracy: 0.4571
Epoch 4/10
1193/1193 [==============================] - 8s 7ms/step - loss: 1.2191 - accuracy: 0.5179
Epoch 5/10
1193/1193 [==============================] - 10s 9ms/step - loss: 1.1709 - accuracy: 0.5480
Epoch 6/10
1193/1193 [==============================] - 9s 7ms/step - loss: 1.1595 - accuracy: 0.5510
Epoch 7/10
1193/1193 [==============================] - 6s 5ms/step - loss: 1.1399 - accuracy: 0.5629
Epoch 8/10
1193/1193 [==============================] - 7s 6ms/step - loss: 1.1325 - accuracy: 0.5736
Epoch 9/10
1193/1193 [==============================] - 5s 5ms/step - loss: 1.1034 - accuracy: 0.5958
Epoch 10/10
1193/1193 [==============================] - 6s 5ms/step - loss: 1.

In [ ]:
test_loss, test_acc = model.evaluate(test_sequences_multi, test_labels, verbose=2)
print('Test accuracy:', test_acc)

299/299 - 1s - loss: 0.9144 - accuracy: 0.7844 - 1s/epoch - 4ms/step
Test accuracy: 0.7843589186668396


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential

# Load the dataset
datamodel2 = pd.read_csv('./cyberbullying_tweets.csv')
datamodel2['cyberbullying_type'] = datamodel2['cyberbullying_type'].replace({'not_cyberbullying':0,'gender':1,'religion':2,'age':3,'ethnicity':4,'other_cyberbullying':5})

# Split the dataset into training and testing sets
train_data = datamodel2.sample(frac=0.8, random_state=42)
test_data = datamodel2.drop(train_data.index)

# Tokenize the text data
tokenizer = Tokenizer(num_words=5000, lower=True)
tokenizer.fit_on_texts(train_data['tweet_text'])
train_sequences = tokenizer.texts_to_sequences(train_data['tweet_text'])
test_sequences = tokenizer.texts_to_sequences(test_data['tweet_text'])

# Pad the sequences to make them of equal length
max_len = 100
train_sequences = pad_sequences(train_sequences, maxlen=max_len)
test_sequences = pad_sequences(test_sequences, maxlen=max_len)

# Define the CNN model
model2 = Sequential()
model2.add(Embedding(input_dim=5000, output_dim=32, input_length=max_len))
model2.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
model2.add(MaxPooling1D(pool_size=2))
model2.add(Flatten())
model2.add(Dense(128, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(6, activation='softmax'))

# Compile the model
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Convert labels to one-hot encoded format
train_labels = tf.keras.utils.to_categorical(train_data['cyberbullying_type'], num_classes=6)
test_labels = tf.keras.utils.to_categorical(test_data['cyberbullying_type'], num_classes=6)
# Train the model
model2.fit(train_sequences, train_labels, epochs=10, batch_size=32)

# Evaluate the model
test_loss, test_acc = model2.evaluate(test_sequences, test_labels, verbose=2)
print('Test accuracy:', test_acc)


Epoch 1/10
1193/1193 [==============================] - 30s 24ms/step - loss: 0.6117 - accuracy: 0.7330
Epoch 2/10
1193/1193 [==============================] - 8s 7ms/step - loss: 0.3747 - accuracy: 0.8435
Epoch 3/10
1193/1193 [==============================] - 6s 5ms/step - loss: 0.3262 - accuracy: 0.8672
Epoch 4/10
1193/1193 [==============================] - 7s 6ms/step - loss: 0.2860 - accuracy: 0.8825
Epoch 5/10
1193/1193 [==============================] - 5s 4ms/step - loss: 0.2600 - accuracy: 0.8919
Epoch 6/10
1193/1193 [==============================] - 8s 7ms/step - loss: 0.2269 - accuracy: 0.9047
Epoch 7/10
1193/1193 [==============================] - 5s 4ms/step - loss: 0.2023 - accuracy: 0.9149
Epoch 8/10
1193/1193 [==============================] - 5s 4ms/step - loss: 0.1791 - accuracy: 0.9248
Epoch 9/10
1193/1193 [==============================] - 6s 5ms/step - loss: 0.1601 - accuracy: 0.9320
Epoch 10/10
1193/1193 [==============================] - 5s 5ms/step - loss: 0.1

In [ ]:
# Train the model
model2.fit(train_sequences, train_labels, epochs=200, batch_size=512)

# Evaluate the model
test_loss, test_acc = model2.evaluate(test_sequences, test_labels, verbose=2)
print('Test accuracy:', test_acc)

Epoch 1/200
75/75 [==============================] - 12s 160ms/step - loss: 0.1093 - accuracy: 0.9515
Epoch 2/200
75/75 [==============================] - 8s 101ms/step - loss: 0.1041 - accuracy: 0.9527
Epoch 3/200
75/75 [==============================] - 6s 79ms/step - loss: 0.0994 - accuracy: 0.9543
Epoch 4/200
75/75 [==============================] - 2s 33ms/step - loss: 0.0967 - accuracy: 0.9551
Epoch 5/200
75/75 [==============================] - 5s 64ms/step - loss: 0.0942 - accuracy: 0.9552
Epoch 6/200
75/75 [==============================] - 3s 40ms/step - loss: 0.0915 - accuracy: 0.9568
Epoch 7/200
75/75 [==============================] - 3s 35ms/step - loss: 0.0892 - accuracy: 0.9570
Epoch 8/200
75/75 [==============================] - 2s 23ms/step - loss: 0.0881 - accuracy: 0.9564
Epoch 9/200
75/75 [==============================] - 2s 22ms/step - loss: 0.0866 - accuracy: 0.9578
Epoch 10/200
75/75 [==============================] - 2s 32ms/step - loss: 0.0857 - accuracy: 0.9

In [ ]:
# Train the model
model2.fit(train_sequences, train_labels, epochs=100, batch_size=64)

# Evaluate the model
test_loss, test_acc = model2.evaluate(test_sequences, test_labels, verbose=2)
print('Test accuracy:', test_acc)

Epoch 1/100
597/597 [==============================] - 34s 56ms/step - loss: 0.0830 - accuracy: 0.9584
Epoch 2/100
597/597 [==============================] - 7s 12ms/step - loss: 0.0732 - accuracy: 0.9622
Epoch 3/100
597/597 [==============================] - 5s 8ms/step - loss: 0.0620 - accuracy: 0.9636
Epoch 4/100
597/597 [==============================] - 5s 9ms/step - loss: 0.0588 - accuracy: 0.9645
Epoch 5/100
597/597 [==============================] - 4s 7ms/step - loss: 0.0601 - accuracy: 0.9639
Epoch 6/100
597/597 [==============================] - 3s 5ms/step - loss: 0.0616 - accuracy: 0.9638
Epoch 7/100
597/597 [==============================] - 3s 5ms/step - loss: 0.0605 - accuracy: 0.9653
Epoch 8/100
597/597 [==============================] - 3s 5ms/step - loss: 0.0570 - accuracy: 0.9659
Epoch 9/100
597/597 [==============================] - 3s 6ms/step - loss: 0.0592 - accuracy: 0.9645
Epoch 10/100
597/597 [==============================] - 3s 5ms/step - loss: 0.0590 - acc

In [ ]:
# Train the model
model2.fit(train_sequences, train_labels, epochs=100, batch_size=32)

# Evaluate the model
test_loss, test_acc = model2.evaluate(test_sequences, test_labels, verbose=2)
print('Test accuracy:', test_acc)

Epoch 1/100
1193/1193 [==============================] - 5s 4ms/step - loss: 0.0581 - accuracy: 0.9658
Epoch 2/100
1193/1193 [==============================] - 6s 5ms/step - loss: 0.0630 - accuracy: 0.9644
Epoch 3/100
1193/1193 [==============================] - 5s 4ms/step - loss: 0.0592 - accuracy: 0.9658
Epoch 4/100
1193/1193 [==============================] - 6s 5ms/step - loss: 0.0626 - accuracy: 0.9644
Epoch 5/100
1193/1193 [==============================] - 5s 4ms/step - loss: 0.0587 - accuracy: 0.9647
Epoch 6/100
1193/1193 [==============================] - 5s 4ms/step - loss: 0.0599 - accuracy: 0.9645
Epoch 7/100
1193/1193 [==============================] - 6s 5ms/step - loss: 0.0613 - accuracy: 0.9643
Epoch 8/100
1193/1193 [==============================] - 5s 4ms/step - loss: 0.0620 - accuracy: 0.9657
Epoch 9/100
1193/1193 [==============================] - 6s 5ms/step - loss: 0.0563 - accuracy: 0.9653
Epoch 10/100
1193/1193 [==============================] - 6s 5ms/step - l

In [ ]:
#RNN

In [ ]:
rnndata = pd.read_csv('./cyberbullying_tweets.csv',encoding='ISO-8859-1')

In [ ]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download("stopwords")
def data_clean(X):
  ps = PorterStemmer()
  corpus = []
  for i in range(len(X)):
    print(i)
    news = re.sub('[^a-zA-Z]', ' ', X[i])
    news = re.sub(r'http\S+', '', news)
    news = re.sub(r'@\S+', '', news)
    news = re.sub(r'#\S+', '', news)
    news = re.sub(r"(?:\@|https?\://)\S+","",news) #remove links and mentions
    news = re.sub(r'[^\x00-\x7f]',r'',news)
    news = re.sub("\s\s+"," ",news)
    news = news.lower()
    news = news.split()
    news = [ps.stem(word) for word in news if word not in stopwords.words('english')]
    news = ' '.join(news)
    corpus.append(news)
  return corpus



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
rnncorpus = data_clean(rnndata['tweet_text'])

Streaming output truncated to the last 5000 lines.
42692
42693
42694
42695
42696
42697
42698
42699
42700
42701
42702
42703
42704
42705
42706
42707
42708
42709
42710
42711
42712
42713
42714
42715
42716
42717
42718
42719
42720
42721
42722
42723
42724
42725
42726
42727
42728
42729
42730
42731
42732
42733
42734
42735
42736
42737
42738
42739
42740
42741
42742
42743
42744
42745
42746
42747
42748
42749
42750
42751
42752
42753
42754
42755
42756
42757
42758
42759
42760
42761
42762
42763
42764
42765
42766
42767
42768
42769
42770
42771
42772
42773
42774
42775
42776
42777
42778
42779
42780
42781
42782
42783
42784
42785
42786
42787
42788
42789
42790
42791
42792
42793
42794
42795
42796
42797
42798
42799
42800
42801
42802
42803
42804
42805
42806
42807
42808
42809
42810
42811
42812
42813
42814
42815
42816
42817
42818
42819
42820
42821
42822
42823
42824
42825
42826
42827
42828
42829
42830
42831
42832
42833
42834
42835
42836
42837
42838
42839
42840
42841
42842
42843
42844
42845
42846
42847
42848
42849
4

In [ ]:
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping

max_words = 10000
max_len = 100
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(rnncorpus)
sequences = tokenizer.texts_to_sequences(rnncorpus)
word_index = tokenizer.word_index
rnndata_sequence = pad_sequences(sequences, maxlen=max_len)

In [ ]:
model = Sequential()
model.add(Embedding(max_words, 100, input_length=max_len))
model.add(Bidirectional(LSTM(64, recurrent_dropout=0.2)))
model.add(Dense(1, activation='sigmoid'))


In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
rnndata['cyberbullying_type'] = rnndata['cyberbullying_type'].apply(lambda x: 0 if x=='not_cyberbullying' else 1)
print(rnndata.head(2))

                                          tweet_text  cyberbullying_type
0  In other words #katandandre, your food was cra...                   0
1  Why is #aussietv so white? #MKR #theblock #ImA...                   0


In [ ]:
from sklearn.model_selection import train_test_split
rnnXtrain,rnnXtest,rnnYtrain,rnnYtest = train_test_split(rnndata_sequence,rnndata['cyberbullying_type'],test_size=0.2,random_state=42)
rnnXtrain,rnnXval,rnnYtrain,rnnYval = train_test_split(rnnXtrain,rnnYtrain,test_size=0.2,random_state=42)

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=3)
history = model.fit(rnnXtrain, rnnYtrain, epochs=10, batch_size=512, validation_data=(rnnXval, rnnYval), callbacks=[early_stop])

Epoch 1/10
60/60 [==============================] - 48s 805ms/step - loss: 0.0910 - accuracy: 0.9538 - val_loss: 0.6265 - val_accuracy: 0.8500
Epoch 2/10
60/60 [==============================] - 45s 752ms/step - loss: 0.0848 - accuracy: 0.9555 - val_loss: 0.6685 - val_accuracy: 0.8506
Epoch 3/10
60/60 [==============================] - 47s 783ms/step - loss: 0.0805 - accuracy: 0.9569 - val_loss: 0.7181 - val_accuracy: 0.8505
Epoch 4/10
60/60 [==============================] - 49s 801ms/step - loss: 0.0780 - accuracy: 0.9583 - val_loss: 0.7367 - val_accuracy: 0.8459


In [ ]:
test_loss, test_acc = model.evaluate(rnnXtest,rnnYtest)
print('Test accuracy:', test_acc)

299/299 [==============================] - 18s 59ms/step - loss: 0.7903 - accuracy: 0.8378
Test accuracy: 0.8378236889839172


In [ ]:
#Rnn multiclassification


In [ ]:
rnnmultidata = pd.read_csv('./cyberbullying_tweets.csv',encoding='ISO-8859-1')
rnnmultidata.head(2)

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying


In [ ]:
#rnnmultidata['cyberbullying_type'] = rnnmultidata['cyberbullying_type'].replace({'not_cyberbullying':0,'gender':1,'religion':2,'age':3,'ethnicity':4,'other_cyberbullying':5})
rnnmultidata.head(2)

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying


In [ ]:
rnnmulticorpus = data_clean(rnnmultidata['tweet_text'])

Streaming output truncated to the last 5000 lines.
42692
42693
42694
42695
42696
42697
42698
42699
42700
42701
42702
42703
42704
42705
42706
42707
42708
42709
42710
42711
42712
42713
42714
42715
42716
42717
42718
42719
42720
42721
42722
42723
42724
42725
42726
42727
42728
42729
42730
42731
42732
42733
42734
42735
42736
42737
42738
42739
42740
42741
42742
42743
42744
42745
42746
42747
42748
42749
42750
42751
42752
42753
42754
42755
42756
42757
42758
42759
42760
42761
42762
42763
42764
42765
42766
42767
42768
42769
42770
42771
42772
42773
42774
42775
42776
42777
42778
42779
42780
42781
42782
42783
42784
42785
42786
42787
42788
42789
42790
42791
42792
42793
42794
42795
42796
42797
42798
42799
42800
42801
42802
42803
42804
42805
42806
42807
42808
42809
42810
42811
42812
42813
42814
42815
42816
42817
42818
42819
42820
42821
42822
42823
42824
42825
42826
42827
42828
42829
42830
42831
42832
42833
42834
42835
42836
42837
42838
42839
42840
42841
42842
42843
42844
42845
42846
42847
42848
42849
4

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
max_words = 10000
max_len = 100
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(rnnmulticorpus)
sequences_multi = tokenizer.texts_to_sequences(rnnmulticorpus)
word_index = tokenizer.word_index
data = pad_sequences(sequences_multi, maxlen=max_len)

# Encode labels
encoder = LabelEncoder()
encoder.fit(rnnmultidata['cyberbullying_type'])
encoded_labels = encoder.transform(rnnmultidata['cyberbullying_type'])
num_classes = len(encoder.classes_)
one_hot_labels = to_categorical(encoded_labels, num_classes=num_classes)

# Split data into training, validation, and test sets
rnnmultiXtrain, rnnmultiXtest, rnnmultiYtrain, rnnmultiYtest = train_test_split(data, one_hot_labels, test_size=0.2, random_state=42)
rnnmultiXtrain, rnnmultiXval, rnnmultiYtrain, rnnmultiYval = train_test_split(rnnmultiXtrain, rnnmultiYtrain, test_size=0.2, random_state=42)

# Create model
model2rnn = Sequential()
model2rnn.add(Embedding(max_words, 100, input_length=max_len))
model2rnn.add(Bidirectional(LSTM(64)))
model2rnn.add(Dense(num_classes, activation='softmax'))

model2rnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=3)
history = model2rnn.fit(rnnmultiXtrain, rnnmultiYtrain, epochs=10, batch_size=512, validation_data=(rnnmultiXval, rnnmultiYval))#, callbacks=[early_stop])

Epoch 1/10
60/60 [==============================] - 2s 30ms/step - loss: 0.1197 - accuracy: 0.9494 - val_loss: 0.6292 - val_accuracy: 0.8125
Epoch 2/10
60/60 [==============================] - 2s 31ms/step - loss: 0.1149 - accuracy: 0.9508 - val_loss: 0.6473 - val_accuracy: 0.8159
Epoch 3/10
60/60 [==============================] - 2s 40ms/step - loss: 0.1091 - accuracy: 0.9515 - val_loss: 0.6598 - val_accuracy: 0.8108
Epoch 4/10
60/60 [==============================] - 2s 42ms/step - loss: 0.1046 - accuracy: 0.9522 - val_loss: 0.6984 - val_accuracy: 0.8113
Epoch 5/10
60/60 [==============================] - 3s 49ms/step - loss: 0.1002 - accuracy: 0.9544 - val_loss: 0.7301 - val_accuracy: 0.8106
Epoch 6/10
60/60 [==============================] - 2s 31ms/step - loss: 0.0974 - accuracy: 0.9544 - val_loss: 0.7546 - val_accuracy: 0.8034
Epoch 7/10
60/60 [==============================] - 2s 34ms/step - loss: 0.0943 - accuracy: 0.9552 - val_loss: 0.7664 - val_accuracy: 0.8057
Epoch 8/10
60

In [ ]:
test_loss2, test_acc2 = model2rnn.evaluate(rnnmultiXtest, rnnmultiYtest)
print('Test accuracy:', test_acc2)

299/299 [==============================] - 2s 6ms/step - loss: 0.8934 - accuracy: 0.7920
Test accuracy: 0.7920117378234863


In [ ]:
#BERT

In [5]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 82.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.2 MB/s eta 0:00:00


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import BertTokenizer, TFBertForSequenceClassification,TFBertModel


In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bertmodel = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
#binary
bertdata = pd.read_csv('/content/drive/MyDrive/Minor Project(NLP)/cyberbullying_tweets.csv',encoding='ISO-8859-1')
bertdata.head()

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


In [ ]:
bertdatabinary=bertdata.copy()
print(bertdatabinary.head(2))

                                          tweet_text cyberbullying_type
0  In other words #katandandre, your food was cra...  not_cyberbullying
1  Why is #aussietv so white? #MKR #theblock #ImA...  not_cyberbullying


In [ ]:
bertdatabinary['cyberbullying_type'] = bertdatabinary['cyberbullying_type'].apply(lambda x: 0 if x=='not_cyberbullying' else 1)
bertdatabinary.head(2)

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",0
1,Why is #aussietv so white? #MKR #theblock #ImA...,0


In [ ]:
texts = bertdatabinary['tweet_text'].values
labels = bertdatabinary['cyberbullying_type'].values
input_ids=[]
attention_masks=[]

In [ ]:
for text in texts:
    encoded_dict = tokenizer.encode_plus(
                        text,                      # text to encode
                        add_special_tokens = True, # add [CLS] and [SEP]
                        max_length = 64,           # truncate long sentences
                        pad_to_max_length = True,  # pad shorter sentences
                        return_attention_mask = True, # create attention mask
                        return_tensors = 'tf',     # return TensorFlow tensors
                   )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])
input_ids = tf.concat(input_ids, axis=0)
attention_masks = tf.concat(attention_masks, axis=0)
labels = tf.constant(labels)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
print(input_ids.shape,np.asarray(input_ids))
#print(attention_masks.shape,attention_masks.type())
#print(labels.shape,labels.type())


(47692, 64) [[  101  1999  2060 ...     0     0     0]
 [  101  2339  2003 ...     0     0     0]
 [  101  1030  1060 ...     0     0     0]
 ...
 [  101  1045  8415 ...     0     0     0]
 [  101  6300  2050 ...     0     0     0]
 [  101 22953  1012 ...     0     0     0]]


In [ ]:
train_inputs, test_inputs, train_labels, test_labels = train_test_split(np.asarray(input_ids), np.asarray(labels), test_size=0.2, random_state=42)
train_masks, test_masks, _, _ = train_test_split(np.asarray(attention_masks), np.asarray(labels), test_size=0.2, random_state=42)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-8)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
bertmodel.compile(optimizer=optimizer, loss=loss, metrics=[metric])
bertmodel.fit(x=(train_inputs, train_masks), y=train_labels, batch_size=32, epochs=3, validation_data=((test_inputs, test_masks), test_labels))

Epoch 1/3
1193/1193 [==============================] - 582s 442ms/step - loss: 0.2637 - accuracy: 0.8731 - val_loss: 0.2335 - val_accuracy: 0.8889
Epoch 2/3
1193/1193 [==============================] - 509s 427ms/step - loss: 0.2032 - accuracy: 0.9039 - val_loss: 0.2307 - val_accuracy: 0.8939
Epoch 3/3
1193/1193 [==============================] - 508s 426ms/step - loss: 0.1594 - accuracy: 0.9282 - val_loss: 0.2644 - val_accuracy: 0.8844


In [ ]:
pred_labels = bertmodel.predict(x=(test_inputs, test_masks))
pred_labels = tf.argmax(pred_labels.logits, axis=1).numpy()
accuracy = accuracy_score(test_labels, pred_labels)
precision = precision_score(test_labels, pred_labels)
recall = recall_score(test_labels, pred_labels)
f1 = f1_score(test_labels, pred_labels)
print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1-score:', f1)

299/299 [==============================] - 51s 143ms/step
Accuracy: 0.884369430757941
Precision: 0.9080996884735203
Recall: 0.9575489576753
F1-score: 0.9321689932968452


In [ ]:
#multiclass classification


In [9]:
multibertdata = bertdata.copy()
multibertdata['cyberbullying_type'] = multibertdata['cyberbullying_type'].replace({'not_cyberbullying':0,'gender':1,'religion':2,'age':3,'ethnicity':4,'other_cyberbullying':5})
multibertdata.head(2)

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",0
1,Why is #aussietv so white? #MKR #theblock #ImA...,0


In [10]:
input_ids_multi = []
attention_masks_multi = []
for tweet in multibertdata['tweet_text']:
    encoded_dict = tokenizer.encode_plus(
                        tweet,
                        add_special_tokens = True,
                        max_length = 128,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'tf'
                   )
    input_ids_multi.append(encoded_dict['input_ids'])
    attention_masks_multi.append(encoded_dict['attention_mask'])
input_ids_multi = tf.concat(input_ids_multi, axis=0)
attention_masks_multi = tf.concat(attention_masks_multi, axis=0)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [11]:
print(input_ids_multi.shape)
print(attention_masks_multi.shape)


(47692, 128)
(47692, 128)


In [12]:
train_input_ids, test_input_ids, train_attention_masks, test_attention_masks, train_labels, test_labels = train_test_split(np.asarray(input_ids_multi), np.asarray(attention_masks_multi), multibertdata['cyberbullying_type'], test_size=0.2, random_state=42)

In [13]:
multi_bert_model = TFBertModel.from_pretrained('bert-base-uncased', output_hidden_states=False, output_attentions=False)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [14]:
input_ids_multi = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name="input_ids")
attention_masks_multi = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name="attention_masks")
bert_output = multi_bert_model([input_ids_multi, attention_masks_multi])[1]
dense_layer_1 = tf.keras.layers.Dense(256, activation='relu')(bert_output)
dropout_layer = tf.keras.layers.Dropout(0.1)(dense_layer_1)
output_layer = tf.keras.layers.Dense(6, activation='softmax')(dropout_layer)
multimodel = tf.keras.models.Model(inputs=[input_ids_multi, attention_masks_multi], outputs=output_layer)

In [ ]:
optimizer = tf.keras.optimizers.legacy.Adam(lr=2e-5, epsilon=1e-08, decay=0.01)
multimodel.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


In [ ]:
history = multimodel.fit(
            [train_input_ids, train_attention_masks],
            train_labels,
            epochs=3,
            batch_size=32,
            validation_split=0.1
         )


Epoch 1/3
1074/1074 [==============================] - 866s 787ms/step - loss: 0.5191 - accuracy: 0.7947 - val_loss: 0.3882 - val_accuracy: 0.8433
Epoch 2/3
1074/1074 [==============================] - 852s 793ms/step - loss: 0.3777 - accuracy: 0.8493 - val_loss: 0.3711 - val_accuracy: 0.8485
Epoch 3/3
1074/1074 [==============================] - 852s 793ms/step - loss: 0.3560 - accuracy: 0.8595 - val_loss: 0.3609 - val_accuracy: 0.8530


In [ ]:
test_loss, test_acc = multimodel.evaluate([test_input_ids, test_attention_masks], test_labels)
print('Test accuracy:', test_acc)

299/299 [==============================] - 80s 269ms/step - loss: 0.3738 - accuracy: 0.8527
Test accuracy: 0.852709949016571


In [15]:
cd /content/drive/MyDrive/Minor Project(NLP)

/content/drive/MyDrive/Minor Project(NLP)


In [ ]:
#fine-tuning BERT model
learning_rate = [2e-5,3e-5,5e-5]
batches=[16,32]
epochs = [2,3,4]
for i in epochs:
  for j in batches:
      optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-8)
      loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
      metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
      bertmodel.compile(optimizer=optimizer, loss=loss, metrics=[metric])
      bertmodel.fit(x=(train_inputs, train_masks), y=train_labels, batch_size=j, epochs=i, validation_data=((test_inputs, test_masks), test_labels))
      pred_labels = bertmodel.predict(x=(test_inputs, test_masks))
      pred_labels = tf.argmax(pred_labels.logits, axis=1).numpy()
      accuracy = accuracy_score(test_labels, pred_labels)
      precision = precision_score(test_labels, pred_labels)
      recall = recall_score(test_labels, pred_labels)
      f1 = f1_score(test_labels, pred_labels)
      print('---For Epoch ',i,'Batch_size',j,'---')
      print('Accuracy:', accuracy)
      print('Precision:', precision)
      print('Recall:', recall)
      print('F1-score:', f1)
      mdict = {'Learning_Rate':[2e-5],'Epoch':[i],'Batch-Size':[j],'Accuracy':[accuracy],'Precision' :[precision],'Recall':[recall],'F1-score':[f1]}
      mdictpd = pd.DataFrame(mdict)
      mdictpd.to_csv('Tuning.csv',mode='a',index='False',header='False')
      print('Appended to csv file')

Epoch 1/2
2385/2385 [==============================] - 672s 258ms/step - loss: 0.1553 - accuracy: 0.9329 - val_loss: 0.2741 - val_accuracy: 0.8913
Epoch 2/2
299/299 [==============================] - 47s 145ms/step
---For Epoch  2 Batch_size 16 ---
Accuracy: 0.8841597651745466
Precision: 0.8979663394109397
Recall: 0.9706885660138976
F1-score: 0.9329123914759273
Appended to csv file
Epoch 1/2
1193/1193 [==============================] - 596s 461ms/step - loss: 0.0906 - accuracy: 0.9587 - val_loss: 0.3786 - val_accuracy: 0.8786
Epoch 2/2
299/299 [==============================] - 46s 146ms/step
---For Epoch  2 Batch_size 32 ---
Accuracy: 0.8703218366705106
Precision: 0.897594665396523
Recall: 0.9523689197725838
F1-score: 0.924170906638877
Appended to csv file
Epoch 1/3
2385/2385 [==============================] - 692s 271ms/step - loss: 0.0722 - accuracy: 0.9635 - val_loss: 0.4430 - val_accuracy: 0.8658
Epoch 2/3
2385/2385 [==============================] - 631s 265ms/step - loss: 0.0664

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



---For Epoch  4 Batch_size 32 ---
Accuracy: 0.8614110493762449
Precision: 0.917860311826594
Recall: 0.9148452305748579
F1-score: 0.916350291065553
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-31-bc8e6b850b85>", line 25, in <cell line: 5>
    mdictpd.to_csv('Tuning.csv',mode='a',index='False',header='False')
  File "/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py", line 211, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pandas/core/generic.py", line 3720, in to_csv
    return DataFrameRenderer(formatter).to_csv(
  File "/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py", line 211, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py", line 1189, in to_csv
    csv_formatt

In [19]:
mdict = {'Learning_Rate':[2e-5],'Epoch':[0],'Batch-Size':[0],'Accuracy':[1]}
mdictpd = pd.DataFrame(mdict)
mdictpd.to_csv('multiTuning.csv')

In [ ]:
batches=[16,32]
epochs = [2,3,4]
for i in epochs:
  for j in batches:
      if i==2 and j==16 : pass
      optimizer = tf.keras.optimizers.legacy.Adam(lr=2e-5, epsilon=1e-08, decay=0.01)
      multimodel.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
      history = multimodel.fit(
            [train_input_ids, train_attention_masks],
            train_labels,
            epochs=i,
            batch_size=j,
            validation_split=0.1
         )
      test_loss, test_acc = multimodel.evaluate([test_input_ids, test_attention_masks], test_labels)
      print('Test accuracy for epoch',i,'batch size',j,'is', test_acc)
      mdict = {'Learning_Rate':[2e-5],'Epoch':[i],'Batch-Size':[j],'Accuracy':[test_acc]}
      mdictpd = pd.DataFrame(mdict)
      mdictpd.to_csv('multiTuning.csv',mode='a',index='False',header='False')
      print('Appended to csv file')

Epoch 1/2
2147/2147 [==============================] - 939s 429ms/step - loss: 0.5200 - accuracy: 0.7927 - val_loss: 0.4121 - val_accuracy: 0.8336
Epoch 2/2
299/299 [==============================] - 83s 277ms/step - loss: 0.4050 - accuracy: 0.8422
Test accuracy for epoch 2 batch size 16 is 0.8422266244888306
Appended to csv file
Epoch 1/2
1074/1074 [==============================] - 908s 829ms/step - loss: 0.3712 - accuracy: 0.8505 - val_loss: 0.3588 - val_accuracy: 0.8559
Epoch 2/2
299/299 [==============================] - 84s 280ms/step - loss: 0.3651 - accuracy: 0.8572
Test accuracy for epoch 2 batch size 32 is 0.8572177290916443
Appended to csv file
Epoch 1/3
2147/2147 [==============================] - 944s 432ms/step - loss: 0.3181 - accuracy: 0.8731 - val_loss: 0.3351 - val_accuracy: 0.8632
Epoch 2/3
2147/2147 [==============================] - 922s 430ms/step - loss: 0.2805 - accuracy: 0.8898 - val_loss: 0.3375 - val_accuracy: 0.8653
Epoch 3/3
299/299 [=======================

In [ ]:
batches=[16,32]
epochs = [4]
for i in epochs:
  for j in batches:
      optimizer = tf.keras.optimizers.legacy.Adam(lr=2e-5, epsilon=1e-08, decay=0.01)
      multimodel.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
      history = multimodel.fit(
            [train_input_ids, train_attention_masks],
            train_labels,
            epochs=i,
            batch_size=j,
            validation_split=0.1
         )
      test_loss, test_acc = multimodel.evaluate([test_input_ids, test_attention_masks], test_labels)
      print('Test accuracy for epoch',i,'batch size',j,'is', test_acc)
      mdict = {'Learning_Rate':[2e-5],'Epoch':[i],'Batch-Size':[j],'Accuracy':[test_acc]}
      mdictpd = pd.DataFrame(mdict)
      mdictpd.to_csv('multiTuning.csv',mode='a',index='False',header='False')
      print('Appended to csv file')